<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/Kaggle_Toxic_Comments_Feb19/Model-3-keras_lstmConv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:
import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D, Input, SpatialDropout1D, Embedding, Dense, Activation, Bidirectional, LSTM,CuDNNLSTM, CuDNNGRU, GlobalMaxPooling1D,GlobalAveragePooling1D, concatenate, Dropout
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.engine import Layer
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from torch.optim.optimizer import Optimizer
from unidecode import unidecode

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/kaggle_toxic_comments"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
best_model.hdf5				test.csv
ext_fasttextwiki_embeddings.npy		test_features.npy
ext_glove_embeddings.npy		test_labels.csv
ext_lexvec_embeddings.npy		toxic_lookup_en_wiki_fasttext_300d.npy
features.npy				train.csv
inputed_lexvec_embeddings.npy		word_index.npy
local_fasttextwiki_embeddings.npy	x_test.npy
sample_submission.csv			x_train.npy
submission_baseline_lr.csv		y_train.npy
submission_model1_Feb21_auc_985062.csv


In [0]:
#configs
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector) ##sum([word[1] for word in vocab[:120000]])=92%##
maxlen = 250 # max number of words in a question to use ## train.num_words.quantile(q=0.96, interpolation='nearest')=258 ##
batch_size = 512 # how many samples to process at once
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits

SEED = 2019
def seed_everything(seed=2019):
    random.seed(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [0]:
os.chdir("/content/drive/My Drive/kaggle_toxic_comments/")
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
features = np.load("features.npy")
test_features = np.load("test_features.npy")
word_index = np.load("word_index.npy").item()
ext_lexvec_embeddings = np.load("ext_lexvec_embeddings.npy")

In [0]:
# Train simple LSTM using different embeddings
# Top compeitions- QIQC and Toxic comments
# ensembles (stacking and blending)
# Pretrained models- ULMFit and BERT
# Gensim and fasttext text classification
# Other SOTAs

In [0]:
##Model-3: https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/80568

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size = 0.1)

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

            
file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

In [0]:
inp = Input(shape=(maxlen,))
x   = Embedding(max_features, embed_size, weights=[ext_lexvec_embeddings], trainable=False)(inp)
x   = SpatialDropout1D(rate=0.2)(x)
x   = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
x   = Conv1D(64, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x)
x1  = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x1   = Conv1D(64, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x1)
x_max   = GlobalMaxPooling1D()(x)
x_avg   = GlobalAveragePooling1D()(x)
x1_max  = GlobalMaxPooling1D()(x1)
x1_avg  = GlobalAveragePooling1D()(x1)
c   = concatenate([x_max,x_avg,x1_max,x1_avg])
y   = Dense(64, activation='relu')(c)
y   = Dropout(0.2)(y)
out = Dense(6, activation='sigmoid')(y)
model=Model(inputs=inp,outputs=out)
model.compile(loss='binary_crossentropy', optimizer=(lr=1e-3), metrics=["accuracy"])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 250, 300)     36000000    input_4[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_4 (SpatialDro (None, 250, 300)     0           embedding_4[0][0]                
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 250, 256)     330240      spatial_dropout1d_4[0][0]        
__________________________________________________________________________________________________
conv1d_5 (

In [0]:
model.fit(X_train, Y_train, batch_size = 128, epochs = 6, validation_data = (X_valid, Y_valid), verbose = 1, callbacks = [ra_val, check_point, early_stop])


Train on 143613 samples, validate on 15958 samples
Epoch 1/3
143613/143613 [==============================] - 289s 2ms/step - loss: 0.0444 - acc: 0.9828 - val_loss: 0.0422 - val_acc: 0.9833

 ROC-AUC - epoch: 1 - score: 0.985711

Epoch 00001: val_loss improved from 0.04301 to 0.04221, saving model to best_model.hdf5
Epoch 2/3
143613/143613 [==============================] - 285s 2ms/step - loss: 0.0426 - acc: 0.9833 - val_loss: 0.0413 - val_acc: 0.9839

 ROC-AUC - epoch: 2 - score: 0.986680

Epoch 00002: val_loss improved from 0.04221 to 0.04130, saving model to best_model.hdf5
Epoch 3/3
143613/143613 [==============================] - 284s 2ms/step - loss: 0.0409 - acc: 0.9841 - val_loss: 0.0416 - val_acc: 0.9831

 ROC-AUC - epoch: 3 - score: 0.988075

Epoch 00003: val_loss did not improve from 0.04130


In [0]:
pred = model.predict(x_test, batch_size = 1024, verbose = 1)

153164/153164 [==============================] - 52s 341us/step


In [0]:
submission = pd.read_csv("./sample_submission.csv")
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = (pred)
submission.to_csv("submission_model2_Feb22_auc_988075.csv", index = False)

In [0]:
##Next tries:
#https://www.kaggle.com/tunguz/bi-gru-cnn-poolings-gpu-kernel-version
#https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/80568 